# 量的データと質的データの可視化



今回は、文化庁の[メディア芸術データベース・ラボ（MADB Lab）](https://mediag.bunka.go.jp/madb_lab/)で公開されている四大少年誌（週刊少年サンデー、週刊少年ジャンプ、週刊少年チャンピオン、週刊少年マガジン）のデータを使って、量的データと質的データの可視化を練習します。

まず、「四大少年誌それぞれの掲載作品のジャンルと著者にはどのような特徴があるのか？」という大きな問いを立て、可視化手法を学びながらデータを見て、具体的な問いを決めていきましょう。

[マンガと学ぶデータビジュアライゼーション](https://kakeami.github.io/viz-madb/index.html)の内容を参考にしています。

ここではPlotlyというライブラリを使います。MatplotlibとAltairの中間のような書き方ができるライブラリですが、Altairほど明示的にデータと視覚記号、データ変数と視覚変数の対応関係をとりません。

書き方は難しくないので、以下のコードを読んで把握しておきましょう。

今日の演習では、以下の中から好きな図を2つ選んで、Altairで再現してもらいます。

## ライブラリの読み込み

In [59]:
import pandas as pd
import numpy as np

import altair as alt
import plotly.express as px

In [60]:
import itertools
import warnings
warnings.filterwarnings('ignore')

## 準備関数

In [61]:
def add_years_to_df(df, unit_years=10):
    """unit_years単位で区切ったyears列を追加"""
    df_new = df.copy()
    df_new['years'] = \
        pd.to_datetime(df['datePublished']).dt.year \
        // unit_years * unit_years
    df_new['years'] = df_new['years'].astype(str)
    return df_new

In [62]:
def resample_df_by_cname_and_years(df):
    """cnameとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    cnames = df['cname'].unique()
    for cname, years in itertools.product(cnames, yearss):
        df_tmp = df_new[
            (df_new['cname'] == cname)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'cname': cname,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = df_new.append(
                s, ignore_index=True)
    return df_new

In [63]:
def resample_df_by_creator_and_years(df):
    """creatorとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    creators = df['creator'].unique()
    for creator, years in itertools.product(creators, yearss):
        df_tmp = df_new[
            (df_new['creator'] == creator)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'creator': creator,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = df_new.append(
                s, ignore_index=True)
    return df_new

## データの用意

四大少年誌の`1970-07-27`から`2017-07-06`までの全ての掲載作品のデータを使います。

すでに前処理がされているデータがあるので、ありがたく使わせていただきます。

In [64]:
file = "https://raw.githubusercontent.com/shinchu/dataviz-notebooks/main/data/week_4/episodes.csv"

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.shape

各週の掲載作品を一行ずつ格納しているため、合計で約18万行程度になります。

In [ ]:
df.columns

- `mcname`: 雑誌名（**M**gazine **C**ollection **NAME**）
- `miid`：雑誌巻号ID（**M**agazine **I**tem **ID**）
- `miname`: 雑誌巻号名（**M**agazine **I**tem **NAME**）
- `cid`: マンガ作品ID（**C**omic **ID**）
- `cname`: マンガ作品名（**C**omic **NAME**）
- `epname`: 各話タイトル（**EP**isode **NAME**）
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の出版社
- `editor`: 雑誌の編集者（編集長）
- `pages`: 各話のページ数（`pageEnd` - `pageStart` + 1）
- `pageEndMax`: 雑誌に掲載されているマンガ作品のうち，`pageEnd`の最大値
- `pageStartPosition`: 各話の`pageStart`の相対的な位置（`pageStart` / `pageEndMax`）

In [13]:
df.head()

,mcname,miid,miname,cid,cname,epname,creator,pageStart,pageEnd,numberOfPages,datePublished,price,publisher,editor,pages,pageEndMax,pageStartPosition
0,週刊少年ジャンプ,M616363,週刊少年ジャンプ 1970年 表示号数31,C88180,男一匹ガキ大将,土佐の源蔵の巻,本宮ひろ志,7.0,37.0,280.0,1970-07-27,80.0,集英社,長野規,31.0,275.0,0.025455
1,週刊少年チャンピオン,M558279,週刊少年チャンピオン 1970年 表示号数14,C94272,朝日の恋人,NaN,かざま鋭二,15.0,43.0,292.0,1970-07-27,80.0,秋田書店 ∥ アキタショテン,成田清美,29.0,290.0,0.051724
2,週刊少年ジャンプ,M616363,週刊少年ジャンプ 1970年 表示号数31,C87448,ど根性ガエル,男はつらいよの巻,吉沢やすみ,39.0,53.0,280.0,1970-07-27,80.0,集英社,長野規,15.0,275.0,0.141818
3,週刊少年チャンピオン,M558279,週刊少年チャンピオン 1970年 表示号数14,C94289,あばしり一家,アバシリ吉三の美人地獄編,永井豪,48.0,66.0,292.0,1970-07-27,80.0,秋田書店 ∥ アキタショテン,成田清美,19.0,290.0,0.165517
4,週刊少年ジャンプ,M616363,週刊少年ジャンプ 1970年 表示号数31,C88021,あらし!三匹,ミヒル登場の巻,池沢さとし,56.0,70.0,280.0,1970-07-27,80.0,集英社,長野規,15.0,275.0,0.203636


In [14]:
df.describe()

,pageStart,pageEnd,numberOfPages,price,pages,pageEndMax,pageStartPosition
count,179931.000000,179931.000000,179624.000000,179893.000000,179931.000000,179931.000000,179931.000000
mean,210.845105,228.371240,417.376325,203.871740,18.526135,408.249379,0.514837
std,123.860878,122.038708,67.418723,41.955489,7.712730,69.804101,0.283146
min,1.000000,1.000000,36.000000,80.000000,1.000000,200.000000,0.002045
25%,107.000000,126.000000,356.000000,180.000000,17.000000,346.000000,0.274840
50%,205.000000,222.000000,437.000000,210.000000,19.000000,433.000000,0.520588
75%,305.000000,322.000000,464.000000,236.000000,20.000000,457.000000,0.759626
max,581.000000,600.000000,600.000000,371.000000,487.000000,600.000000,1.000000


欠損値を確認してみます。

特に`epname`と`publisher`の欠測が多いことがわかります．

In [15]:
df.isna().sum().reset_index()

,index,0
0,mcname,0
1,miid,0
2,miname,0
3,cid,9
4,cname,9
5,epname,26807
6,creator,441
7,pageStart,0
8,pageEnd,0
9,numberOfPages,307


## 量を見る

### 棒グラフ

棒グラフ（Bar Chart） は、主に質的変数を対象にして、棒の長さで数量を表すグラフです。棒を縦方向に並べることもありますし、横方向に並べることもあります。質的変数の量を見る最も一般的な方法の一つです。

#### 作品別の掲載週数（上位20作品）

In [36]:
# 作品ごとの週数を数える
df_plot = df.value_counts("cname").reset_index(name="weeks").head(20)

Altairで再現する

In [58]:
alt.Chart(df_plot).mark_bar().encode(
    alt.X("cname:N", sort="-y", title="作品名"),
    alt.Y("weeks:Q", title="掲載週数"),
    alt.Tooltip(["weeks"])
).properties(
    title="作品ごとの掲載週数"
).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=16
)

alt.Chart(...)

#### 作品別・年代別の掲載週数（上位20作品）

In [55]:
# dfに10年区切りの年代情報を追加
df = add_years_to_df(df)

In [ ]:
df_plot = df.groupby()

## 分布を見る

## 比率を見る

## 変数の関係を見る